In [1]:
!pip install segmentation_models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 6.2 MB/s 


In [ ]:
!pip install keras_unet_collection

In [ ]:
# 기본 모듈 임포트
import glob
import shutil
import os
import random
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 이미지 관련 모듈 임포트
import cv2
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array

# 모델 관련 모듈 임포트
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Activation, Flatten, Add, Dropout, ReLU, Conv2D, MaxPooling2D, UpSampling2D, concatenate

# 손실함수 임포트
from segmentation_models.losses import bce_dice_loss
from keras_unet_collection import losses

In [ ]:
# 평가 함수
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

---
### 데이터 로드
---
> data 아래의 디렉터리인 test, train을 불러옵니다.

In [ ]:
# 증강시킨 데이터를 가져온다
X_train, X_test, y_train, y_test = np.load('./data/data256_FRC+_5000.npy', allow_pickle=True)

# 효율적인 학습을 위해 스케일링한다
X_train = X_train/255.
X_test = X_test/255.
y_train = y_train/255.
y_test = y_test/255.

---
### 예측
---

In [ ]:
# 모델을 가져온다.
model = load_model('./model/last_model_unet_5000_hybrid.h5', custom_objects = {'binary_crossentropy_plus_dice_loss' : bce_dice_loss})

In [ ]:
# 학습된 결과를 출력후 비교
random_test = random.sample(range(X_test.shape[0]),7)  # 테스트할 이미지를 랜덤하게 불러옵니다.
ae_imgs = model.predict(X_test)                     # 앞서 만든 모델에 집어 넣습니다.

# 학습 이미지 후처리 후 리스트에 저장
size = 256
thresh_list = []
at_ae_imgs = ae_imgs.copy()
for idx, save_img in enumerate(at_ae_imgs) :
  np.putmask(at_ae_imgs[idx],at_ae_imgs[idx] < 0.5, 0)
  np.putmask(at_ae_imgs[idx],at_ae_imgs[idx] >= 0.5, 1)
  thresh_list.append(img_to_array(at_ae_imgs[idx]))
# 넘파이 배열로 변경
thresh_list = np.array(thresh_list)

# 이미지 출력후 비교
plt.figure(figsize=(20, 10))                         # 출력될 이미지의 크기를 정합니다.
for i, image_idx in enumerate(random_test):        # 랜덤하게 뽑은 이미지를 차례로 나열합니다.
   ax = plt.subplot(4, 7, i + 1) 
   plt.imshow(X_test[image_idx].squeeze(), cmap='gray')   # 테스트할 이미지를 먼저 그대로 보여줍니다.
   ax.axis('off')
   ax = plt.subplot(4, 7, 7 + i +1)
   plt.imshow(ae_imgs[image_idx].squeeze(), cmap='gray')  # 예측 결과를 다음열에 출력합니다.
   ax.axis('off')
   ax = plt.subplot(4, 7, 7*2 + i +1)
   plt.imshow(thresh_list[image_idx].squeeze(), cmap='gray')  # 후처리한 이미지를 다음열에 출력합니다.
   ax.axis('off')
   ax = plt.subplot(4, 7, 7*3 + i +1)
   plt.imshow(y_test[image_idx].squeeze(), cmap='gray')  # Label을 다음열에 출력합니다.
   ax.axis('off')
plt.show()

In [ ]:
# 실험 결과
print('점수')
print('다이스 점수 : ', dice_coef(y_test, ae_imgs.squeeze()).numpy())
print('IoU 점수', 1 -losses.iou_seg(y_test, ae_imgs.squeeze()).numpy())
print('='*70)
print('후처리 점수')
print('다이스 점수 : ', dice_coef(y_test, thresh_list.squeeze()).numpy())
print('IoU 점수', 1 -losses.iou_seg(y_test, thresh_list.squeeze()).numpy())